In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
trips_df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [3]:
trips_df['duration'] = trips_df['tpep_dropoff_datetime'] - trips_df['tpep_pickup_datetime']
trips_df.duration = trips_df.duration.apply(lambda td: td.total_seconds() / 60)

trips_df = trips_df[(trips_df.duration >= 1) & (trips_df.duration <= 60)]

In [4]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

trips_df[categorical] = trips_df[categorical].astype(str)

In [5]:
train_dicts = trips_df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = trips_df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649143388169879

In [6]:
val_df = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
val_df['duration'] = val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']
val_df.duration = val_df.duration.apply(lambda td: td.total_seconds() / 60)

val_df = val_df[(val_df.duration >= 1) & (val_df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

val_df[categorical] = val_df[categorical].astype(str)

val_dicts = val_df[categorical + numerical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

target = 'duration'
y_val = val_df[target].values

In [7]:
y_pred = lr.predict(X_val)

In [8]:
mean_squared_error(y_val, y_pred, squared=False)

7.811472580390592